## Import Library

In [ ]:
!pip install kbbi

In [ ]:
# import kbbi library
from kbbi import KBBI
from kbbi import TidakDitemukan
from kbbi import AutentikasiKBBI

In [ ]:
# Import Library Python
import numpy as np
import pandas as pd
import json

## Loading Data

Memuat dataset berisi aspek.

## Dataset Unique Aspects

Menyaring aspek-aspek yang unik dalam dataset ulasan.

In [ ]:
file_newset = f'https://docs.google.com/spreadsheets/d/1EBJ9tWAzQHfOop_URWD-vieipDtWtIZIxIWjlI4-VwU/gviz/tq?tqx=out:csv'
all_new = pd.read_csv(file_newset, usecols=['review', 'aspect', 'sentiment'])

In [ ]:
all_new

,review,aspect,sentiment
0,tempat yang sakral sekali dibali setelah pura ...,tempat,0
1,tempat yang sakral sekali dibali setelah pura ...,lokasi,-1
2,tempat yang sakral sekali dibali setelah pura ...,lokasi,1
3,ini tempat taman wisata air paling lengkap di ...,taman wisata air,1
4,ini adalah pengalaman yang luar biasa. hotel m...,pengalaman,1
...,...,...,...
7985,"pemandangan yang sangat sangat menawan, kombin...",tiket,1
7986,"pemandangan yang sangat sangat menawan, kombin...",parkir,1
7987,dinner with the great elephant adalah salah sa...,great elephant,1
7988,dinner with the great elephant adalah salah sa...,program,1


In [ ]:
aspects = pd.DataFrame(data=all_new['aspect'].copy())
aspects.drop_duplicates(inplace=True)
aspects.reset_index(drop=True, inplace=True)
aspects

,aspect
0,tempat
1,lokasi
2,taman wisata air
3,pengalaman
4,hotel
...,...
1735,program
1736,sawah bersusun
1737,model slide
1738,karang pantai


In [ ]:
aspects.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1740 entries, 0 to 1739
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   aspect  1740 non-null   object
dtypes: object(1)
memory usage: 13.7+ KB


## sample dataset aspek

Pengambilan data dari KBBI Daring sekaligus berpotensi menyebabkan akun KBBI dibekukan. Untuk menghindari hal tersebut, dataset aspek dibagi menjadi batch-batch kecil. Pengambilan pengetahuan dari KBBI dilakukan per subdataset.

In [ ]:
file_un_asp2 = f'https://docs.google.com/spreadsheets/d/1XDV1M5_hrWlGFKix8c2PgPmpuZFj3b9xrI-UtYmOiZM/gviz/tq?tqx=out:csv'
un_asp2 = pd.read_csv(file_un_asp2, usecols=['index', 'aspect', 'knowledge'])
un_asp2.dropna(subset="aspect", inplace=True)
un_asp2

,index,aspect,knowledge
0,478,kaus,NaN
1,482,loket,NaN
2,483,pohon,NaN
3,484,patung,NaN
4,485,kandang,NaN
...,...,...,...
95,695,menginap,NaN
96,697,keadaan,NaN
97,698,bus,NaN
98,700,susunan,NaN


In [ ]:
un_asp2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   index      100 non-null    int64  
 1   aspect     100 non-null    object 
 2   knowledge  0 non-null      float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.5+ KB


In [ ]:
un_asp2_split = np.array_split(un_asp2, 10)
un_asp2_split

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


[   index     aspect  knowledge
 0    478       kaus        NaN
 1    482      loket        NaN
 2    483      pohon        NaN
 3    484     patung        NaN
 4    485    kandang        NaN
 5    489  kaki lima        NaN
 6    491  pemesanan        NaN
 7    495   panorama        NaN
 8    499       bule        NaN
 9    500   berjalan        NaN,
     index        aspect  knowledge
 10    504     bersantai        NaN
 11    505      berjemur        NaN
 12    510        budaya        NaN
 13    516        klinik        NaN
 14    517     paramedik        NaN
 15    520     penjagaan        NaN
 16    522         kapal        NaN
 17    523        safari        NaN
 18    525    ketertiban        NaN
 19    526  persimpangan        NaN,
     index      aspect  knowledge
 20    527   pedangang        NaN
 21    528     pinggir        NaN
 22    529     sekitar        NaN
 23    531    olahraga        NaN
 24    532      rumput        NaN
 25    534       jalur        NaN
 26    536  

In [ ]:
un_asp2_split[2]

,index,aspect,knowledge
20,527,pedangang,NaN
21,528,pinggir,NaN
22,529,sekitar,NaN
23,531,olahraga,NaN
24,532,rumput,NaN
25,534,jalur,NaN
26,536,spa,NaN
27,541,burung,NaN
28,543,berfoto,NaN
29,549,kereta api,NaN


# Knowledge Retrieval

Mendefinisikan fungsi untuk mengambil pengetahuan dari KBBI.

In [ ]:
def retrieve_knowledge(kata):
  try:
    kbbi_find = KBBI(kata)
  except TidakDitemukan as e:
    kbbi_find = e.objek
  kjdata = json.loads(json.dumps(kbbi_find.serialisasi()))
  makna = []
  for i in range(0,len(kjdata['entri'])):
    for j in range(0,len(kjdata['entri'][i]['makna'])):
      makna.append(kjdata['entri'][i]['makna'][j]['submakna'][0])
  knowledge = " [SEP] ".join(makna)
  return knowledge

In [ ]:
retrieve_knowledge("tempat")

'sesuatu yang dipakai untuk menaruh (menyimpan, meletakkan, dan sebagainya) [SEP] ruang (bidang, rumah, dan sebagainya) yang tersedia untuk melakukan sesuatu [SEP] ruang (bidang dan sebagainya) yang dipakai untuk menaruh (menyimpan, mengumpulkan, dan sebagainya) [SEP] ruang (bidang, rumah, daerah, dan sebagainya) yang didiami (ditinggali) atau ditempati [SEP] bagian yang tertentu dari suatu ruang (bidang, daerah, dan sebagainya) [SEP] negeri (kota, desa, daerah, dan sebagainya) [SEP] sesuatu yang dapat (dipercaya) menampung (tentang isi hati, keluhan, pertanyaan, dan sebagainya) [SEP] kedudukan'

### processing aspects_split2[7]

In [ ]:
un_asp2_split[9]

,index,aspect,knowledge
90,682,menunggang,NaN
91,684,pijat,NaN
92,685,medan,NaN
93,686,matahari terbit,NaN
94,691,lukisan,NaN
95,695,menginap,NaN
96,697,keadaan,NaN
97,698,bus,NaN
98,700,susunan,NaN
99,702,lalu lintas,NaN


In [ ]:
un_asp2_split[9]['knowledge'] = un_asp2_split[9].aspect.apply(lambda x: retrieve_knowledge(x))

In [ ]:
un_asp2_split[9]

,index,aspect,knowledge
90,682,menunggang,"menaiki [SEP] menggunakan (gerakan, perkumpula..."
91,684,pijat,tekan [SEP] urut [SEP] cari: pijat-pijat
92,685,medan,tanah lapang [SEP] lingkungan [SEP] lokasi [SE...
93,686,matahari terbit,matahari hidup
94,691,lukisan,hasil melukis [SEP] cerita atau uraian yang me...
95,695,menginap,"menumpang tidur (di rumah orang, di hotel, dan..."
96,697,keadaan,sifat [SEP] suasana
97,698,bus,"kendaraan bermotor angkutan umum yang besar, b..."
98,700,susunan,sesuatu yang sudah diatur (disusun dan sebagai...
99,702,lalu lintas,(berjalan) bolak-balik [SEP] perihal perjalana...


# Combine and save splitted dataset

In [ ]:
un_asp2_combine = pd.concat(un_asp2_split)
un_asp2_combine

,index,aspect,knowledge
0,478,kaus,NaN
1,482,loket,NaN
2,483,pohon,NaN
3,484,patung,NaN
4,485,kandang,NaN
...,...,...,...
95,695,menginap,"menumpang tidur (di rumah orang, di hotel, dan..."
96,697,keadaan,sifat [SEP] suasana
97,698,bus,"kendaraan bermotor angkutan umum yang besar, b..."
98,700,susunan,sesuatu yang sudah diatur (disusun dan sebagai...


In [ ]:
from google.colab import files

un_asp2_combine.to_csv("un_asp2_combine.csv")
files.download("un_asp2_combine.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>